In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

from finrock.data_feeder import PdDataFeeder
from finrock.trading_env import TradingEnv
from finrock.render import PygameRender

# The following allows to save plots in SVG format.
import matplotlib_inline
%matplotlib inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [5]:
df = pd.read_csv('Datasets/random_sinusoid.csv')
df = df[-1000:]

model_path = "runs/1715936222"

pd_data_feeder = PdDataFeeder.load_config(df, model_path)
env = TradingEnv.load_config(pd_data_feeder, model_path)

action_space = env.action_space
input_shape = env.observation_space.shape
pygameRender = PygameRender(frame_rate=30)#120

agent = tf.keras.models.load_model(f'{model_path}/ppo_sinusoid_actor.h5')

state, info = env.reset()
pygameRender.render(info)
rewards = 0.0
while True:
    # simulate model prediction, now use random action
    prob = agent.predict(np.expand_dims(state, axis=0), verbose=False)[0]
    action = np.argmax(prob)

    state, reward, terminated, truncated, info = env.step(action)
    rewards += reward
    pygameRender.render(info)

    if terminated or truncated:
        print(rewards)
        for metric, value in info['metrics'].items():
            print(metric, value)
        state, info = env.reset()
        rewards = 0.0
        pygameRender.reset()
        pygameRender.render(info)


UnboundLocalError: cannot access local variable 'prev_value_map' where it is not associated with a value